In [46]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn import manifold, datasets
from sklearn.model_selection import train_test_split
from time import time
from grakel.kernels import NeighborhoodSubgraphPairwiseDistance
from grakel.kernels import WeisfeilerLehman
from grakel.kernels import ShortestPath
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy import sparse, io
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mat4py
import glob
import pandas as pd
from grakel import Graph

In [47]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [48]:
path = 'PPI'
files = glob.glob(path + '/*.csv')

lists = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=None)
    lists.append(df)
    
frame = pd.concat(lists, axis=0, ignore_index=True)
data_ = clean_dataset(frame)

height, width = data_.shape
n_samples = labels.shape[0]
print("n_samples", n_samples)

labels = pd.read_csv('ppi_labels.csv').T
# labels = labels.values.flatten()
# print(labels.T.shape)

# data_, labels = data_.align(labels, axis=1, fill_value=0)

labels = labels.T
print(labels.shape)
print(data_.shape)


n_samples 85
(85, 1)
(232, 232)


In [49]:
print("Splitting dataset into train/test (1000/100 instances)")
graphs_train, graphs_test = data_[:150], data_[150:232]
y_train, y_test = labels[:57], labels[57:85]

# # split a dataset into train and test sets
# from sklearn.datasets import make_blobs
# from sklearn.model_selection import train_test_split
# # create dataset
# X, y = make_blobs(n_samples=1000)
# split into train test sets
# graphs_train, graphs_test, y_train, y_test = train_test_split(data_, labels, test_size=0.33)
print(graphs_train.shape, graphs_test.shape, y_train.shape, y_test.shape)

Splitting dataset into train/test (1000/100 instances)
(150, 232) (82, 232) (57, 1) (28, 1)


In [51]:
gk = ShortestPath(normalize=True)
K_train = gk.fit_transform(graphs_train)
K_test = gk.transform(graphs_test)

# Uses the SVM classifier to perform classification
clf = SVC(kernel="precomputed")
clf.fit(K_train, y_train)
y_pred = clf.predict(K_test)

# Computes and prints the classification accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", str(round(acc*100, 2)) + "%")

TypeError: each element of X must have at least one and at most 3 elements


In [44]:
graphs = list()
edges = list()
edge_labels = dict()
for i in range(height):
    for j in range(width):
        if j < width-1:
            edges.append((i*height+j, i*height+j+1))
            edge_labels[(i*height+j, i*height+j+1)] = 1
        if i < height-1:
            edges.append((i*height+j, (i+1)*height+j))
            edge_labels[(i*height+j, (i+1)*height+j)] = 2

for i in range(n_samples):
    node_labels = dict()
    for j in range(height):
        for k in range(width):
            node_labels[j*height+k] = int(labels[i,j]/4)
            
            
# for i in range(len(doc)):
#             for j in range(i+1, i+window_size):
#                 if j < len(doc):
#                     G.add_edge(doc[i], doc[j])
    
    graphs.append(Graph(edges, node_labels=node_labels, edge_labels=edge_labels))


KeyError: (0, 0)

In [10]:
# Initialize a Weisfeiler-Lehman subtree kernel
gk = WeisfeilerLehman(n_iter=1, normalize=False)

# Construct kernel matrices
K_train = gk.fit_transform(graphs_train)
print(K_trainain)
K_test = gk.transform(graphs_test)

# Train an SVM classifier and make predictions
clf = SVC(kernel='precomputed')
clf.fit(K_train, y_train) 
y_pred = clf.predict(K_test)

# Evaluate the predictions
print("Accuracy:", accuracy_score(y_pred, y_test))

TypeError: each element of X must be either a graph object or a list with at least a graph like object and node labels dict 


In [7]:
# Initialize neighborhood subgraph pairwise distance kernel
gk = NeighborhoodSubgraphPairwiseDistance(r=3, d=2)

print("Computing kernel matrics\n")
t0 = time()
print(graphs_train)
K_train = gk.fit_transform(graphs_train)
K_test = gk.transform(graphs_test)
print("done in %0.3fs\n" % (time() - t0))

print("Classifying digits\n")
# Initialize SVM
clf = SVC(kernel='precomputed')

# Fit on the train Kernel
clf.fit(K_train, y_train)

# Predict and test.
y_pred = clf.predict(K_test)

# Calculate accuracy of classification.
print("Classification accuracy: %0.2f" % accuracy_score(y_test, y_pred))

Computing kernel matrics

      0    1    2    3    4    5    6    7    8    9    ...  222  223  224  \
2909  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  ...  1.0  0.0  0.0   
2910  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2911  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2912  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2913  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
3054  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3055  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3056  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
3057  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3058  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

      225  226  227  228  229  230  2

TypeError: each element of X must have either a graph with labels for node and edge or 3 elements consisting of a graph type object, labels for vertices and labels for edges.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
sample_submission = pd.read_csv("../input/sampleSubmission.csv")
training_labels = LabelEncoder().fit_transform(train['target'])

# SVMs tend to like features that look similar to ~ N(0,1), so let's stabilise
# the long tails
train_features = train.drop('target', axis=1)
train_features[train_features > 4] = 4

model = LinearSVC().fit(train_features, training_labels)

scores = model.decision_function(test)
predictions = 1.0 / (1.0 + np.exp(-scores))
row_sums = predictions.sum(axis=1)
predictions_normalised = predictions / row_sums[:, np.newaxis]

# create submission file
prediction_DF = pd.DataFrame(predictions_normalised, index=sample_submission.id.values, columns=sample_submission.columns[1:])
prediction_DF.to_csv('svc_submission.csv', index_label='id')